In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate



chat = ChatOpenAI(
  temperature=0.1,
  streaming=True,
  callbacks=[
    StreamingStdOutCallbackHandler(),
  ],
)


intro = PromptTemplate.from_template(
  """
  You are a role playing assistant.
  And you are impersnating a {character}
"""
)

example = PromptTemplate.from_template(
  """
  This is an example of how you talk :

  Human : {example_question}
  You : {example_answer}
"""
)

start = PromptTemplate.from_template(
  """
  Start!

  Human : {question}
  You
"""
)

final = PromptTemplate.from_template(
  """
  {intro}

  {example}

  {start}
"""
)

prompts = [
  ("intro", intro),
  ("example", example),
  ("start", start)
]

full_prompt = PipelinePromptTemplate(final_prompt=final, 
                                     pipeline_prompts=prompts,)

chain = full_prompt | chat

chain.invoke({
  "character":"Teacher", 
  "example_question":"what is you dream?",
  "example_answer":"My deream is to be a doctor",
  "question":"What is your favorite food?"
  })

: My favorite food is sushi. I love the combination of fresh fish and rice, it's just so delicious! What about you, what's your favorite food?

AIMessageChunk(content=": My favorite food is sushi. I love the combination of fresh fish and rice, it's just so delicious! What about you, what's your favorite food?")